<a href="https://colab.research.google.com/github/PyMap/AUPY/blob/master/Modulo%201/01_0_Introduccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción al manejo de datos geográficos

### Preparándonos para trabajar con geopandas

<figure>
<center>
<img src='../imagenes/caras_pandas.png' />
<figcaption></figcaption></center>
</figure>

## Introducción 

Si llegaste hasta acá, es que seguramente ya contás con los rudimentos básicos para manejar datos con python. Ya manipulaste algunos de sus objetos nativos como listas, diccionarios y tuplas, e incluso trabajaste con arrays de numpy y dataframes de pandas. Bueno, entonces ya contamos con una gran ventaja. 

Si bien es cierto que la espacialidad es un atributo que requiere del conocimiento de algunos métodos específicos (que iremos viendo a lo largo del modulo), también lo es que los `datos espaciales` no dejan de ser `datos`. En otras palabras, esto significa que para familiarizarnos con ellos apelaremos a muchas de las estretegias que seguramente ya forman parte de tu repertortio. 

De esta manera, la **propuesta para esta primera clase** será la siguiente:

**0)** *Repaso general*: A lo largo de esta introducción, revisaremos algunos recursos de gran utilidad para el tratamiento de nuestros datos. Fijaremos los conceptos que están por detrás de métodos y funciones mayormemnte utilizados para trabajar con distintos tipos de datos: map y apply, diferencias entre funciones regulares y anónimas, etc. serán el principal objetivo de este repaso. Todo esto, con la finalidad de saltar al mundo de los atributos geo en la primera parte de esta clase.

**1)** *Geopandas*: Se estudiarán los conceptos más relevantes de la librería Geopandas. Entre ellos, los cambios sufridos en los objetos de tipo CRS a partir de la release 0.7.0, reproyecciones, shapely y los métodos aplicables a los distintos tipos de geometrías, joins espaciales y algunas nociones generales de ploteo y geolocalización. El objetivo de esta primera clase será dar un pantallazo general a estos temas, los cuales se irán profundizando a lo largo de las otras clases del módulo. Empecemos! 

### Algunas cuestiones básicas: funciones y métodos. 

Hasta acá, sabemos que una función es un bloque de código que comienza con la sentencia `def` y lleva un nombre asociado. Puede recibir argumentos, los que comúnmente son utilizados en una expresión o secuencia de sentencias para realizar alguna tarea o devolver un objeto. De esta manera...

In [1]:
# Armemos una función que devuelva la suma de los elementos de un iterable...
def sumar(arg):
    '''
    Suma de los items de un objeto iterable.
    ...
    Argumentos:
        arg(iterable): iterable de elementos numéricos 
                       (e.g. 'list' o 'pandas serie')
    Devuelve:
        integer: resultado de la sumatoria
    '''
    total = 0
    for i in range(len(arg)):
        total = total + arg[i] 
    return total

In [2]:
iterable = [2,4,6,8]

In [3]:
sumar(iterable)

20

Vemos que esta función nos sirve para obtener el resultado. Llamándola y pasándole el parámetro que deseemos nos devuelve la sumatoria de todos sus elementos. Ahora, ¿la forma de ejecutar una función solamente es llamándola de este modo? No, hay otra manera de llamar a las funciones y esta es a través de los métodos. Un `método` es en sí mismo una función pero cuya cualidad principal es la de aplicarse sobre un objeto mismo, utilizando parámetros opcionales. Un método sólo existe dentro del objeto y por eso se lo puede llamar desde sí mismo.

En la POO, las clases tienen `atributos` y `métodos`. Por decirlo de alguna manera, la clase calculadora, tiene números y los puede sumar. En python, los métodos se definen dentro de las clases y se llaman desde el objeto aplicando un paréntesis. Esta es la otra manera de llamar funciones que nos estaba faltando. Veámosla...

In [4]:
# llamemos la función que creamos enteriormente...
sumar(iterable)

20

In [5]:
# hagamos lo mismo, sumar todos los elementos de un iterable pero ahora usando numpy...
import numpy as np

In [6]:
# hasta ahora tenemos el mismo resultado
np.sum(iterable)

20

Tratemos ahora de sumar los elementos de nuestra lista pero pensando a la suma como una propiedad que esta tiene y puede aplicar sobre sí misma. Es decir como si puedese hacerlo por el hecho de ser una lista:

In [7]:
# veamos qué sucede
iterable.sum()

AttributeError: 'list' object has no attribute 'sum'

Queda claro por el error. La lista no cuenta con un método integrado que permita automáticamente sumar los elementos que la componen. Pero no ocurre lo mismo con los objetos de numpy. Continuemos con el ejemplo y veamos por qué...

In [8]:
# convertimos nuesta lista en un array de numpy
np.array(iterable).sum()

20

Vemos que llamando a la función suma como un método del objeto ya no nos devuelve un error. Y esto es porque los arrays de numpy sí cuentan con un método o función integrada que les permite sumar los elementos que los componen (como así también restarlos, conseguir un promedio, y otras cosas con las que python nativo no cuenta. A menos que escribamos una función que lo haga como hicimos al principio de este ejemplo!).

Ahora que ya revisamos esto, tenemos en claro que los métodos se llaman o ejecutan desde el objeto mismo mientras que las funciones lo hacen por fuera y aplicando parámetros a una expresión. 

Revisemos ahora algunas formas alternativas de aplicar funciones y de llamar métodos que nos serán realmente útiles.

### Compañeros de ruta: la función lambda y los métodos map(), apply() y applymap() 

A esta altura, esperamos que algunas de las herramientas que has venido utilizando casi `de memoria` empiecen a decantar por si solas. Con seguridad necesitaste agregar o transformar columnas a tus dataframes para que cumplieran con determinadas características. Y para ello, debes haber utilizado métodos y funciones que nunca terminaste de entender bien qué es lo que hacían. Bueno, acá vamos a hacer una revisión rápida para terminar de entender cómo se usan y cuál es su equivalencia con estructuras de datos nativas de python. 

Hicimos esta selección porque creemos que son las de uso más frecuente y te servirán para terminar de redondear y afianzar muchas de las cosas que ya venís haciendo. Pensá esto como una cáscara que, a medida que avancemos, iremos llenando con atributos y métodos propios del mundo `geo`.

In [9]:
# empecemos por importar pandas con su clásico alias
import pandas as pd

> **1. La función anónima `lambda`**

Es muy útil pensar esta función como si el **`lambda`** equivaliera al **`def`** en una función regular. Para ponerlo en otros términos, digamos que la manera en la que definimos una función anónima también nos permite crear parámetros que vamos a usar para aplicar una expresión a partir de la que transformaremos algún objeto. 

<figure>
<center>
<img src='../imagenes/sintaxis_funciones_anonima_regular.jpg' />
<figcaption></figcaption></center>
</figure>

Esto, con dos diferencias esenciales. **La primera, que la función anónima no requiere de un `return` para devolvernos un resultado**. 

<figure>
<center>
<img src='../imagenes/funcionamiento_anonima_regular.jpg' />
<figcaption></figcaption></center>
</figure>

Veamos algún ejemplo concreto. Para ello, supongamos que contamos con un código (algo bastante común para reconocer unidades administrativas o físicas en un territorio dado) y que queremos hacer alguna transformación sobre el mismo. Por ejemplo, completar su formato para contar con mayor información o hacer algún matcheo o unión con fuentes de información externas. Imaginemos que estamos trabajando con el código postal y que sólo contamos con una de las siguientes secciones...

Vamos un caso real, el del CPA. El mismo está compuesto por ocho caracteres:
1. Una letra identificatoria de la provincia.
2. Un número de 4 dígitos que identifica la localidad, ciudad o barrio.
3. Una combinación de tres letras que identifican la "cara" de la manzana.

<figure>
<center>
<img src='../imagenes/estructura_codigos_postales.png' />
<figcaption></figcaption></center>
</figure>

¿Cómo podríamos completarlo con otras secciones utilizando una función anónima si sólo contaramos con la intermedia, por ejemplo? Veamos cómo...

In [10]:
# instanciamos nuestro código incompleto.
codigo_postal = '1024'

# definimos una función para formar códigos postales que estén en una misma jurisdicción y cara de manzana
completar_formato = lambda x: 'C'+ x +'FDA'

# la aplicamos
completar_formato(codigo_postal)

'C1024FDA'

Como pudimos ver, `x` es nuestro parámetro, aquel sobre el cual se aplicará la expresión delimitada a partir del `lambda`.

**La segunda diferencia, es que las funciones anónimas sólo trabajan con una expresión en su cuerpo** (mientras que las regulares pueden contener muchas). `x` (o como nosotros decidamos que se llame) es lo que se conoce como un `place holder`. Es decir, podemos aplicar la expresión definida en la función a filas de una serie de pandas, a items de una lista, etc. Ahora bien, la función lambda sólo toma una expresión a la vez. Es decir, que debemos aplicarla a tantos objetos o elementos como parámetros hayamos definido en nuestra función.

Pero no nos dejemos confundir. Veamos qué pasa si queremos completar una serie o una lista.

In [11]:
# definimos nuevamente nuestro código
codigo_postal = pd.Series(['C','1024','FDA'])

# rehacemos la función anónima, ahora agregando algunos parámetros más después de la key 'lambda'
completar_formato = lambda x,y,z: x + y + z

In [12]:
# prestemos atención al tipo de error que nos devuelve
completar_formato(codigo_postal)

TypeError: <lambda>() missing 2 required positional arguments: 'y' and 'z'

Como se puede leer, hay dos argumentos que aparecen como missing. Y esto es porque estamos aplicando una función con tres parametros sobre una sola serie. Distinto hubiese sido si...

In [13]:
# instanciamos por separado las distintas parte del código
parte_uno, parte_dos, parte_tres = 'C','1024','FDA'

# y aplicamos la función sobre las tres partes
completar_formato(parte_uno, parte_dos, parte_tres)

'C1024FDA'

O si...

In [14]:
# suponiendo que tenemos una serie con muchas secciones de codigos postales ordenadas
codigo_postal = pd.Series(['C','1024','FDA','C','1171','ABM','C','1097','AAX'])

In [15]:
# rearmamos nuestra función lambda con un solo parametro
completar_formato = lambda x: [x[i]+x[i+1]+x[i+2] for i in x.index if len(x[i])<2]

In [16]:
# para obtener todos los codigos postales con las secciones que estaban dispersas, ahora concatendas
completar_formato(codigo_postal)

['C1024FDA', 'C1171ABM', 'C1097AAX']

> **2. El método `map`**

Este método se utiliza para aplicar una función a todos los elementos de un iterable especificado. De esta manera, su sintáxis queda definida como:

<figure>
<center>
<img src='../imagenes/sintaxis_map.jpg' />
<figcaption></figcaption></center>
</figure>

Se pueden pasar múltiples argumentos, siempre que la función que se esté mapeando tenga esa misma cantidad de argumentos. La función se va a aplicar a dichos iterables en paralelo y se va a detener cuando el iterable más corto haya sido agotado. Veamos algunos ejemplos...

In [17]:
# armamos una funcion regular que sume tres variables
def combina(a,b,c):
    return a+b+c

# aplicamos la función a las tres secciones de nuestro código postal
map(combina,parte_uno, parte_dos, parte_tres)

Detengámonos brevemente en qué nos devuelve, un obtejo `map`. Este es un [`iterador`](https://docs.python.org/3/glossary.html#term-iterator) y en python está diseñado así por una cuestión de eficiencia. Es decir, para evitar guardar en memoria todo sobre lo que se itera, map devuelve un objeto de tipo iterador que se puede visualizar cuando le aplicamos algún contenedor, como una lista o una serie. 

In [18]:
list(map(combina,[parte_uno], [parte_dos], [parte_tres]))

['C1024FDA']

In [19]:
pd.Series(map(combina,[parte_uno],[parte_dos], [parte_tres]))

0    C1024FDA
dtype: object

Y por qué se puede leer cuando le aplicamos un contenedor?, basicamente porque el objeto de tipo `map` es un objeto iterable, es decir, que se puede iterar sobre él:

In [20]:
for i in map(combina, [parte_uno], [parte_dos], [parte_tres]):
    print(i)

C1024FDA


Esto significa que, por ejemplo, si aplcaramos un `list()` a un `map(func, iter)` estaría sucediendo algo como esto:

In [21]:
codigo = []
for i in map(combina,[parte_uno], [parte_dos], [parte_tres]):
    codigo.append(i)
    
codigo

['C1024FDA']

Este comportamiento hace que `map()` sea más rápido que otras alternativas, por ejemplo una lista por comprensión.

In [22]:
import time

In [23]:
partes = ['C','1024','FDA','C','1171','ABM','C','1097','AAX']

In [24]:
# lista por comprension
inicio = time.time()
resultado = [(partes[i]+partes[i+1]+partes[i+2]) for i in range(len(partes)) if i%3==0]
fin = time.time()

In [25]:
resultado

['C1024FDA', 'C1171ABM', 'C1097AAX']

In [26]:
print('Tardó en ejecutarse %f segundos'%(fin-inicio))

Tardó en ejecutarse 0.000124 segundos


In [27]:
# esto es lo que está sucediendo cuando aplicamos una lista por comprensión
codigos = []

for i in range(len(partes)):
    if (i%3==0):
        codigos.append(partes[i]+partes[i+1]+partes[i+2])

In [28]:
codigos

['C1024FDA', 'C1171ABM', 'C1097AAX']

Si esto mismo lo hacemos utilizando el método `map`, primero debemos crear una función que realice la misma tarea. Recordemos, que este itera sobre cada uno de los elementos de un contenedor (en nuestro ejemplo previo, una lista). Por lo tanto, deberemos adaptar o bien la forma en la que diseñamos nuestra función o bien el objeto sobre el que la aplicamos. Hagamos esto último para  no extendernos demasiado...

In [29]:
def combina_partes(lista):
    '''
    Combina los items de un objeto iterable.
    ...
    Argumentos:
        arg(iterable): iterable de strings 
                       (e.g. 'list' o 'pandas serie')
    Devuelve:
        lista: items combinados
    '''
    codigos=[]
    for i in range(len(lista)):
        if (i%3==0):
            codigos.append(lista[i]+lista[i+1]+lista[i+2])
    return codigos

Reparemos ahora en cómo la vamos a aplicar. Como nuestra función fue pensada para operar sobre una lista como único argumento, no podríamos mapear directamente sobre la lista `partes`. Por qué? básicamente porque sino dicha función se aplicaría sobre cada elemento `lista[0]`, `lista[1]`, `lista[n]`. Entonces la vamos a convertir en lista, para que el primer elemento que mapee el iterador sea nuestra lista de string con las secciones de distintos códigos postales.

In [30]:
# mapeamos la función y accedemos al index 0 del resultado para tener nuestros códigos unificados en un solo lugar
inicio = time.time()
resultado = list(map(combina_partes,[partes]))[0]
fin = time.time()

In [31]:
resultado

['C1024FDA', 'C1171ABM', 'C1097AAX']

In [32]:
# como podemos apreciar, el método map hace la misma tarea en un tiempo menor
print('Tardo en ejecutarse %f' % (fin-inicio))

Tardo en ejecutarse 0.000087


Como vimos hasta acá, el `map` aplica una función a un iterable y devuelve el resultado de haber aplicado dicha función en cada uno de los items de ese iterable. Esto es particularmente útil para realizar operaciones con columnas de un dataframe. En el mundo geo, es muy común querer evaluar resultados en un espacio geográfico. Y para eso, muchas de las operaciones que anteceden a la visualización tienen que ver con transformaciones realizadas en columnas de un geodataframe. Antes de mostrar algo en un mapa, primero definimos cuál es ese indicador o valor que queremos ver. Y para construirlo, es muy probable que llevemos a cabo distintos tipos de operaciones sobre series. Acá es donde el método `map` adquiere una mayor relevancia. Pero antes, aclaremos que su uso en una serie es bastante más sencillo de lo que vimos hasta acá.  

Básicamente, cuando utilizamos el `map` sobre una serie de un dataframe o geodataframe, lo hacemos como si este fuera un método del objeto. Así:

In [33]:
# veamos un ejemplo rápido de cómo mapear una función sobre una serie de valores aleatorios
columna = pd.Series(np.random.choice(100,3))

In [34]:
# Y ahora apliquemos la función (para que sea más interesante, hagamos que sea anónima) con map...
columna.map(lambda x: round(x/2,1))

0    31.5
1    40.5
2    43.0
dtype: float64

Pongamos este ejemplo un poco más en contexto para terminar de entender la utilidad del `map`. Supongamos que en la Ciudad de Buenos Aires se implementó un plan urbano que dictamina la demolición de edificios abandonados por un período de tiempo superior a los 20 años. Y supongamos también que los siguientes códigos postales:

In [35]:
pd.Series(resultado)

0    C1024FDA
1    C1171ABM
2    C1097AAX
dtype: object

... corresponden a edificios que cumplen con dicha condición. Dada la crisis habitacional de la ciudad, para estos casos el nuevo plan urbano, dispone la demolición y posterior construcción de nuevas unidades de habitación que se vuelquen al mercado para aumentar la oferta de vivienda asequible. Para terminar de entender en qué nos puede ayudar el método map, digamos que en estos tres edificios se dispone la construcción de: 

In [36]:
print("%s nuevos departamentos que estarán disponibles en el lapso de 24 meses" % (columna.sum()))

230 nuevos departamentos que estarán disponibles en el lapso de 24 meses


In [37]:
# veamos nuestra tabla inicial...
desarrollos = pd.DataFrame({'id':resultado, 
                           'inicio': [2020,2020,2020], 
                           'unidades': columna,
                           'duracion':[24,24,24]})

In [38]:
desarrollos

,id,inicio,unidades,duracion
0,C1024FDA,2020,63,24
1,C1171ABM,2020,81,24
2,C1097AAX,2020,86,24


Ahora, como también sabemos que el gobierno irá construyendo la misma cantidad de unidades por año, queremos que nuestra tabla de desarrollos adopte un nuevo formato. En este debe quedar representado, además del año de inicio el de finalización y la cantidad de unidades por año.

In [39]:
# Armemos una función que haga lo que indica nuestro hipotético escenario...
def desarrollos_anuales(df, duracion, unidades,idx):
    '''
    Distribuye unidades residenciales a lo largo de
    los años de construcción del proyecto.
    ...
    Argumentos:
        df(dataframe): dataframe de proyectos residenciales 
        duracion(str): nombre de columna
        unidades(str): nombre de columna
        idx(str): nombre de columna
    Devuelve:
        dataframe: unidades anualmente distribuidas
    '''
    
    df['años'] = df[duracion].map(lambda x: x/12)
    df['anuales'] = df[unidades].map(lambda x: x/2)
    
    new_df = np.repeat(df[idx], df['años']).reset_index()
    new_df['inicio'] = 2020
    año = lambda v: [v+1 if i%2!=0 else v for i,v in enumerate(new_df['inicio'])]
    new_df['inicia'] = año(new_df['inicio'])
    new_df['finaliza'] = new_df['inicia'] + 1
    
    oferta = np.repeat(df['unidades'].map(lambda x: x/2),df['años']).\
                       reset_index()['unidades']
    new_df['unidades'] = oferta
    new_df.drop(columns='inicio',inplace=True)
    
    return new_df.iloc[:,1:]

In [40]:
desarrollos_anuales(desarrollos, 'duracion', 'unidades','id')

,id,inicia,finaliza,unidades
0,C1024FDA,2020,2021,31.5
1,C1024FDA,2021,2022,31.5
2,C1171ABM,2020,2021,40.5
3,C1171ABM,2021,2022,40.5
4,C1097AAX,2020,2021,43.0
5,C1097AAX,2021,2022,43.0


De esta manera, ya empezamos a trabajar con lo que nos interesa. Cuando entremos con mayor profundidad al mundo de geopandas, veremos que la materia prima con la que deberemos trabajar para mostrar nuestros resultados son los geodataframes. Por eso, es que nos detendremos un poco más en cómo estos métodos se aplican sobre series comunes y corrientes, o incluso en dataframes. Un poco más de paciencia!

> **3. El método `apply`**

Este método trabaja principalmente sobre series de panda. Al igual que el `map`, toma cada elemento dentro de una serie, o incluso un dataframe, y le aplica una función determinada.

<figure>
<center>
<img src='../imagenes/equivalencias_apply_map.jpg' />
<figcaption></figcaption></center>
</figure>

Siguiendo con el ejemplo anterior, supongamos que ahora necesitamos redondear el stock de unidades que se van a construir. Es decir, necesitamos que nuestras unidades estén expresadas en enteros y no perder información cuando el redondeo elimina los decimales.

In [41]:
da = desarrollos_anuales(desarrollos, 'duracion', 'unidades','id')

In [42]:
da

,id,inicia,finaliza,unidades
0,C1024FDA,2020,2021,31.5
1,C1024FDA,2021,2022,31.5
2,C1171ABM,2020,2021,40.5
3,C1171ABM,2021,2022,40.5
4,C1097AAX,2020,2021,43.0
5,C1097AAX,2021,2022,43.0


In [43]:
def entero(x):
    return int(x)

def resto(x):
    if float(x)-int(x)>0:
        return int(x)+1
    else:
        return int(x)+0

In [44]:
def redondea_unidades(df,col1):
    '''
    Asigna el resto de la división al último año
    de construcción (como entero 1).
    ...
    Argumentos:
        df(dataframe): dataframe de proyectos residenciales 
        col1(str): nombre de columna
    Devuelve:
        lista: lista de int
    
    '''
    valores = []
    for i in df[col1].index:
        if i%2 == 0:
            valores.append(df.loc[i:i,col1].apply(entero).values[0])
        else: 
            valores.append(df.loc[i:i,col1].apply(resto).values[0])
    return valores

In [45]:
da['total'] = redondea_unidades(da,'unidades')

In [46]:
da

,id,inicia,finaliza,unidades,total
0,C1024FDA,2020,2021,31.5,31
1,C1024FDA,2021,2022,31.5,32
2,C1171ABM,2020,2021,40.5,40
3,C1171ABM,2021,2022,40.5,41
4,C1097AAX,2020,2021,43.0,43
5,C1097AAX,2021,2022,43.0,43


De esta forma, vemos que para los casos en los que las unidades cuentan con valores decimales, las funciones que aplicamos recurriendo al método `apply` nos permitieron asignar ese resto al último año de la construcción. Mientras que para aquellos casos en los que el resto era cero no se agregó ninguna unidad adicional.

Veamos cómo funciona ahora el mismo método sobre un conjunto de columnas (o un dataframe). Imaginemos que durante nuestro proceso de trabajo debemos cambiar el mismo valor para más de una serie. Digamos que, por ejemplo, los desarrollos se retrasaron trayendo como consecuencia que no sólo se corra el año de inicio sino también el de finalización...

In [47]:
# Supongamos que ese lapso es de un año solamente. Para ello, creemos una función que aplique sobre nuestras filas.
def agrega_años(x):
    return x+1

# Y ahora apliquemosla sobre ellas...
da[['inicia','finaliza']].apply(agrega_años)

,inicia,finaliza
0,2021,2022
1,2022,2023
2,2021,2022
3,2022,2023
4,2021,2022
5,2022,2023


De esta manera, vemos que `apply` no funciona sólo sobre series aisladas, sino también sobre dataframes. Tal como lo hace...

> **4. El método `applymap`**

Pensemos que puede ser bastante frecuente querer llevar adelante una misma operación para más de una columna en un dataframe. Imaginemos que no sólo debemos transformar dos como en el caso anterior, sino muchas más. Para este tipo de circunstancias, el método `applymap` resulta particularmente útil, ya que el mismo opera sobre el dataframe entero.

Para entender un poco mejor su utilidad, supongamos que ahora contamos con nueva información sobre nuestros desarrollos. Además de la cantidad de unidades, sabemos qué porcentage de ellas corresponde a una tipología determinada. Es decir, sabemos que del total que se construirá anualmente un x% corresponde a unidades 0,1,2,3,4 y 5 ambientes (también podríamos haber dado un rango de metros cuadrados, pero quedemonos con esto).

Veamos entonces cómo podemos valernos de aquellos métodos que operan sobre un dataframe entero, `apply` y `applymap` para convertir nuestra columna de totales en porcentages de cada tipología.

In [48]:
# emprolijemos nuestro df de desarrollos anuales eliminando las columnas que no necesitamos
da.drop(columns='unidades', inplace=True)

In [49]:
da

,id,inicia,finaliza,total
0,C1024FDA,2020,2021,31
1,C1024FDA,2021,2022,32
2,C1171ABM,2020,2021,40
3,C1171ABM,2021,2022,41
4,C1097AAX,2020,2021,43
5,C1097AAX,2021,2022,43


In [50]:
# Creemos nuestro dataframe de tipologías
amb = pd.DataFrame()
for i in range(len(da)):
    amb['{}_amb'.format(i)] = [0]*len(da)

In [51]:
amb

,0_amb,1_amb,2_amb,3_amb,4_amb,5_amb
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0


In [52]:
# Ahora, armemos una lista de porcentages a partir de la que se desagregarán los totales de unidades a construir.
porcentages = [0.10, 0.15, 0.30, 0.35, 0.07, 0.03]

In [53]:
# Asignamos los porcentages a las filas. Con random, hacemos que los items se ordenen de forma aleatoria
import random

for i in amb.index:
    random.shuffle(porcentages) 
    amb.loc[[i],:] = porcentages

In [54]:
amb

,0_amb,1_amb,2_amb,3_amb,4_amb,5_amb
0,0.07,0.03,0.10,0.30,0.15,0.35
1,0.15,0.35,0.03,0.07,0.30,0.10
2,0.15,0.30,0.03,0.35,0.07,0.10
3,0.03,0.35,0.07,0.10,0.15,0.30
4,0.30,0.15,0.03,0.07,0.10,0.35
5,0.10,0.07,0.03,0.30,0.15,0.35


In [55]:
# Creemos una lista con los totales que se vamos a convertir en porcentages
totales = [i for i in da['total']]

Y ahora sí, después de tanta introducción, utilicemos el método `applymap` para multiplicar cada celda por el valor total definido en la lista. Reparemos que para cambiar los valores en nuestro dataframe, estamos indexando cada fila para todo el conjunto de columnas. Por último, también es útil remarcar el uso de la función anónima que vimos al principio del notebook. Tanto el método `applymap` como `apply` se llevan particularmente bien con este recurso. Básicamente porque al utilizarse (normalmente) con un solo argumento (la celda de la Serie o las del DataFrame) esto deja un mayor margen para agregar otros operadores (o argumentos) que si estuviesen definidos dentro de una función regular serían más difíciles de llamar. Pero veámoslo con un ejemplo concreto.

In [56]:
# Indexamos nuestro dataframe y combinamos applymap con una función anónima
for i in range(len(totales)):
    amb.loc[[i],:] = amb.loc[[i],:].applymap(lambda x: x*totales[i])

In [57]:
# Comprobamos que la suma de nuestras filas de el total de unidades de nuestro df de desarrollos anuales
amb.sum(axis=1)

0    31.0
1    32.0
2    40.0
3    41.0
4    43.0
5    43.0
dtype: float64

Ahora, probemos lo mismo con el método `apply` pero tratando de llamar otros argumentos que están definidos dentro de una función regular (algo para lo que `applymap` no es muy dúctil).

In [58]:
# Definamos una función que multiplique nuestros valores
def multiplica(x,y):
    return x*y

In [59]:
# Veamos cómo hacemos para llamar el argumento que no es nuestra 'x'(la celda). Apply lo hace muy sencillo...
for idx in amb.index:
    for elem in range(len(totales)):
        amb.loc[[idx],:].apply(multiplica,y=elem)

In [60]:
# Verifiquemos que nuestros valores suman el total de unidades.
amb.sum(axis=1)

0    31.0
1    32.0
2    40.0
3    41.0
4    43.0
5    43.0
dtype: float64

In [61]:
# Veamos cómo nos quedaría nuestro dataframe de desarrollos anuales...
da_completo = pd.merge(da,amb.applymap(lambda x: round(x,5)),left_index=True, right_index=True)

In [62]:
da_completo

,id,inicia,finaliza,total,0_amb,1_amb,2_amb,3_amb,4_amb,5_amb
0,C1024FDA,2020,2021,31,2.17,0.93,3.10,9.30,4.65,10.85
1,C1024FDA,2021,2022,32,4.80,11.20,0.96,2.24,9.60,3.20
2,C1171ABM,2020,2021,40,6.00,12.00,1.20,14.00,2.80,4.00
3,C1171ABM,2021,2022,41,1.23,14.35,2.87,4.10,6.15,12.30
4,C1097AAX,2020,2021,43,12.90,6.45,1.29,3.01,4.30,15.05
5,C1097AAX,2021,2022,43,4.30,3.01,1.29,12.90,6.45,15.05


In [63]:
def redondea_stock_por_tipologia(df, id_unico, id_columna, columna_stock):
    '''
    Convierte enteros por fila y reasigna la diferencia.
    ...
    Argumentos:
        df(dataframe): dataframe de proyectos residenciales
        id_unico (str): codigo postal del inmueble a contruir
        id_columna (str): nombre de columna
        columna_stock (str): nombre de la columna con la tipologia a construir
    Devuelve:
        lista: lista de int
    
    '''
    decimal = df.loc[df[id_columna] == id_unico, [columna_stock]].iloc[:, 0]
    integer = df.loc[df[id_columna] == id_unico, [columna_stock]].iloc[:, 0].astype(int)
    rest = decimal - integer
    unit = int(round(rest.sum(), 0))

    rows = rest.index.values
    idx = np.random.choice(rows, unit, replace=False)
    integer[idx] = integer[idx]+1
    return integer.values

In [64]:
totales = {}

for tipologia in da_completo.columns[4:]:
    totales[tipologia] = [redondea_stock_por_tipologia(da_completo, i, 'id', tipologia) for 
                          i in da_completo['id'].unique()]

In [65]:
totales

{'0_amb': [array([2, 5]), array([6, 1]), array([13,  4])],
 '1_amb': [array([ 1, 11]), array([12, 14]), array([6, 3])],
 '2_amb': [array([3, 1]), array([2, 2]), array([1, 2])],
 '3_amb': [array([9, 3]), array([14,  4]), array([ 4, 12])],
 '4_amb': [array([ 4, 10]), array([3, 6]), array([5, 6])],
 '5_amb': [array([10,  4]), array([ 4, 12]), array([15, 15])]}

In [66]:
for k in totales.keys():
    totales.update({k:np.concatenate(totales[k])})

In [67]:
totales

{'0_amb': array([ 2,  5,  6,  1, 13,  4]),
 '1_amb': array([ 1, 11, 12, 14,  6,  3]),
 '2_amb': array([3, 1, 2, 2, 1, 2]),
 '3_amb': array([ 9,  3, 14,  4,  4, 12]),
 '4_amb': array([ 4, 10,  3,  6,  5,  6]),
 '5_amb': array([10,  4,  4, 12, 15, 15])}

In [68]:
def ajusta_stock(df,totales):
    '''
    Reasigna enteros por fila para alcanzar
    el valor total de unidades.
    ...
    Argumentos:
        df(dataframe): dataframe de proyectos residenciales
        totales (dict): nuevos totales por tipologia
    Devuelve:
        df: df con totales ajustados por tipologia
    
    '''
    df['nvos_totales'] = pd.DataFrame(totales).sum(axis=1)
    df['diferencia'] = df['total'] - df['nvos_totales']
    tipologias = ['0_amb','1_amb','2_amb','3_amb','4_amb','5_amb']
    df[tipologias] = pd.DataFrame(totales)
    
    for index, row in df.iterrows():
        if row['diferencia'] < 0:
            ambientes = random.choice(tipologias)
            print('Eliminando excedente de unidades en:',ambientes)
            df.loc[index, ambientes] = row[ambientes] + (row['diferencia'])
            #df.loc[index, ambientes] = np.abs(df.loc[index, ambientes])
            
        elif row['diferencia'] > 0:
            ambientes = random.choice(tipologias)
            print('Agregando unidades faltantes en:',ambientes)
            df.loc[index, ambientes] = np.abs(row[ambientes] + (row['diferencia']))
            #df.loc[index, ambientes] = np.abs(df.loc[index, ambientes])

        else:
            print('Sin unidades para ajustar')
            pass
    
    df.drop(columns=['nvos_totales','diferencia'],inplace=True)
    
    return df

In [69]:
da_ajustado = ajusta_stock(da_completo,totales)

Agregando unidades faltantes en: 1_amb
Eliminando excedente de unidades en: 3_amb
Eliminando excedente de unidades en: 4_amb
Agregando unidades faltantes en: 5_amb
Eliminando excedente de unidades en: 4_amb
Agregando unidades faltantes en: 4_amb


In [70]:
da_ajustado

,id,inicia,finaliza,total,0_amb,1_amb,2_amb,3_amb,4_amb,5_amb
0,C1024FDA,2020,2021,31,2,3,3,9,4,10
1,C1024FDA,2021,2022,32,5,11,1,1,10,4
2,C1171ABM,2020,2021,40,6,12,2,14,2,4
3,C1171ABM,2021,2022,41,1,14,2,4,6,14
4,C1097AAX,2020,2021,43,13,6,1,4,4,15
5,C1097AAX,2021,2022,43,4,3,2,12,7,15


In [71]:
tipologias = ['0_amb','1_amb','2_amb','3_amb','4_amb','5_amb']
da_ajustado['total'] == da_ajustado[tipologias].sum(axis=1)

0    True
1    True
2    True
3    True
4    True
5    True
dtype: bool

In [72]:
da_ajustado.to_csv('../data/da_ajustado.csv')